In [1]:
%matplotlib inline
import time
import pandas as pd
import numpy as np
import torch

from os.path import join, expanduser
from os import listdir
from PIL import Image
from torchvision import transforms, models
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter


from tqdm import tqdm
from copy import deepcopy

In [2]:
class SeedlingDataset(Dataset):
    def __init__(self, labels, root_dir, subset=False, transform=None):
        self.labels = labels
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        img_name = self.labels.iloc[idx, 0]
        fullname = join(self.root_dir, img_name)
        image = Image.open(fullname).convert('RGB')
        labels = self.labels.iloc[idx, 2]
        if self.transform:
            image = self.transform(image)
        return image, int(labels)

In [3]:
data_dir = './archive/dataset/'
cache_dir = expanduser(join('.', 'cache'))

image_size = 224
batch_size = 64
# увеличил batch_size
classes = listdir(data_dir + 'train/')
classes = sorted(classes, key=lambda item: (int(item.partition(' ')[0])
                               if item[0].isdigit() else float('inf'), item))
num_to_class = dict(zip(range(len(classes)), classes))
num_to_class

{0: 'Alu_Gobi',
 1: 'Alu_Matar',
 2: 'American_Pancakes',
 3: 'Apple_Pie',
 4: 'Arros_negre',
 5: 'Arroz_con_huevo',
 6: 'Arroz_con_pollo',
 7: 'Avocado_Toast',
 8: 'BLT_Sandwich',
 9: 'Bacon_Egg_and_Cheese_Sandwich',
 10: 'Bagels',
 11: 'Baguette',
 12: 'Baked_Ziti',
 13: 'Banana_Bread',
 14: 'Banana_Split',
 15: 'Barbecue_Ribs',
 16: 'Barfi',
 17: 'Beef_Rendang_(rendang)',
 18: 'Beef_Vindaloo',
 19: 'Beefaroni',
 20: 'Beignet',
 21: 'Beignets',
 22: 'Blanquette_de_Veau',
 23: 'Bocadillo_de_carne',
 24: 'Bocadillo_de_jamon',
 25: 'Bocadillo_de_pollo',
 26: 'Bocadillo_de_queso',
 27: 'Boeuf_Bourguignon',
 28: 'Bouillabaisse',
 29: 'Bread_Pudding',
 30: 'Breakfast_Burrito',
 31: 'Brioche',
 32: 'Brodetto_Di_Pesce',
 33: 'Brownies',
 34: 'Buffalo_Wings',
 35: 'Burrito',
 36: 'Butter_Chicken',
 37: 'Cabrales',
 38: 'California-Style_Pizza',
 39: 'Cannele',
 40: 'Carrot_Halwa',
 41: 'Cassoulet',
 42: 'Chaat_Papri',
 43: 'Cham-Cham',
 44: 'Chana_Masala',
 45: 'Chapati',
 46: 'Chausson_aux_P

In [4]:
train = []
for index, label in enumerate(classes):
    path = data_dir + 'train/' + label + '/'
    for file in listdir(path):
        train.append(['{}/{}'.format(label, file), label, index])
    
train_data = pd.DataFrame(train, columns=['file', 'category', 'category_id',]) 
train_data

,file,category,category_id
0,Alu_Gobi/Image_277.jpg,Alu_Gobi,0
1,Alu_Gobi/Image_136.jpg,Alu_Gobi,0
2,Alu_Gobi/Image_105.jpg,Alu_Gobi,0
3,Alu_Gobi/Image_238.jpg,Alu_Gobi,0
4,Alu_Gobi/Image_433.jpg,Alu_Gobi,0
...,...,...,...
121848,Yakisoba/Image_474.jpg,Yakisoba,307
121849,Yakisoba/Image_39.jpg,Yakisoba,307
121850,Yakisoba/Image_77.jpg,Yakisoba,307
121851,Yakisoba/Image_148.jpg,Yakisoba,307


In [5]:
val = []
for index, label in enumerate(classes):
    path = data_dir + 'val/' + label + '/'
    for file in listdir(path):
        val.append(['{}/{}'.format(label, file), label, index])
    
valid_data = pd.DataFrame(val, columns=['file', 'category', 'category_id',]) 
valid_data

,file,category,category_id
0,Alu_Gobi/Image_178.jpg,Alu_Gobi,0
1,Alu_Gobi/Image_457.JPG,Alu_Gobi,0
2,Alu_Gobi/Image_212.jpg,Alu_Gobi,0
3,Alu_Gobi/Image_40.jpg,Alu_Gobi,0
4,Alu_Gobi/Image_90.jpg,Alu_Gobi,0
...,...,...,...
15228,Yakisoba/Image_5.jpg,Yakisoba,307
15229,Yakisoba/Image_365.jpg,Yakisoba,307
15230,Yakisoba/Image_31.jpg,Yakisoba,307
15231,Yakisoba/Image_147.jpg,Yakisoba,307


In [6]:
test = []
for index, label in enumerate(classes):
    path = data_dir + 'test/' + label + '/'
    for file in listdir(path):
        test.append(['{}/{}'.format(label, file), label, index])
    
test_data = pd.DataFrame(test, columns=['file', 'category', 'category_id',]) 
test_data

,file,category,category_id
0,Alu_Gobi/Image_193.jpg,Alu_Gobi,0
1,Alu_Gobi/Image_63.jpg,Alu_Gobi,0
2,Alu_Gobi/Image_73.jpg,Alu_Gobi,0
3,Alu_Gobi/Image_28.JPG,Alu_Gobi,0
4,Alu_Gobi/Image_175.jpg,Alu_Gobi,0
...,...,...,...
15246,Yakisoba/Image_45.jpg,Yakisoba,307
15247,Yakisoba/Image_26.jpg,Yakisoba,307
15248,Yakisoba/Image_401.jpg,Yakisoba,307
15249,Yakisoba/Image_456.jpg,Yakisoba,307


In [7]:
train_trans = transforms.Compose([
    transforms.RandomSizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

valid_trans = transforms.Compose([
    transforms.Scale(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_set = SeedlingDataset(train_data, data_dir + 'train/', transform = train_trans)
valid_set = SeedlingDataset(valid_data, data_dir + 'val/', transform = valid_trans)
test_set = SeedlingDataset(test_data, data_dir + 'test/', transform = valid_trans)

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader  = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=4)

dataset_sizes = {
    'train': len(train_loader.dataset), 
    'valid': len(valid_loader.dataset)
}

/home/gp/Documents/made_project/env/lib/python3.8/site-packages/torchvision/transforms/transforms.py:697: UserWarning: The use of the transforms.RandomSizedCrop transform is deprecated, please use transforms.RandomResizedCrop instead.
  warnings.warn("The use of the transforms.RandomSizedCrop transform is deprecated, " +
/home/gp/Documents/made_project/env/lib/python3.8/site-packages/torchvision/transforms/transforms.py:210: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


In [8]:
def train_model(dataloaders, model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()

    best_model_wts = deepcopy(model.state_dict())
    best_acc = 0.0
    
    writer = SummaryWriter(f"runs/FOOD/tensorboard")
    step = 0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                # scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode   
                
            running_loss = 0.0
            running_corrects = 0
            running_batch = 0

            # Iterate over data.
            for data in tqdm(dataloaders[phase]):
                # get the inputs
                inputs, labels = data
                labels = labels.view(-1)
                
                # wrap them in Variable
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)
                
                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                # running_loss += loss.data[0]
                running_loss += loss.data.item()
                
                num_correct = torch.sum(preds == labels.data).cpu().numpy().sum()
                running_train_acc = float((preds == labels.data).sum()) / float(inputs.shape[0])
                
                running_corrects += num_correct
                # torch.sum(preds == labels.data)
                running_batch +=1
                
                step += 1
                writer.add_scalar("Tranning Loss", loss, global_step=step)
                writer.add_scalar("Training Accuracy", running_train_acc, global_step=step)    
                writer.add_scalar("Training LR", optimizer.param_groups[0]["lr"], global_step=step)
                
            epoch_loss = running_loss / running_batch
            epoch_acc = running_corrects / dataset_sizes[phase]
            
            if phase == "valid":
                    scheduler.step(epoch_loss)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [9]:
use_gpu = torch.cuda.is_available()
# use_gpu = False

model = models.resnet50(pretrained=False)

#I recommend training with these layers unfrozen for a couple of epochs after the initial frozen training
for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, len(classes))

if use_gpu:
    model = model.cuda()

In [10]:
model.load_state_dict(torch.load("./cache/models/resnet50-new_weights_4.pth"))
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [11]:
for name, param in model.named_parameters():
    if "layer4.2." in name:
        print(name, param.requires_grad)
        param.requires_grad = True

layer4.2.conv1.weight False
layer4.2.bn1.weight False
layer4.2.bn1.bias False
layer4.2.conv2.weight False
layer4.2.bn2.weight False
layer4.2.bn2.bias False
layer4.2.conv3.weight False
layer4.2.bn3.weight False
layer4.2.bn3.bias False


In [12]:
for name, param in model.named_parameters():
    print(name, param.requires_grad)

conv1.weight False
bn1.weight False
bn1.bias False
layer1.0.conv1.weight False
layer1.0.bn1.weight False
layer1.0.bn1.bias False
layer1.0.conv2.weight False
layer1.0.bn2.weight False
layer1.0.bn2.bias False
layer1.0.conv3.weight False
layer1.0.bn3.weight False
layer1.0.bn3.bias False
layer1.0.downsample.0.weight False
layer1.0.downsample.1.weight False
layer1.0.downsample.1.bias False
layer1.1.conv1.weight False
layer1.1.bn1.weight False
layer1.1.bn1.bias False
layer1.1.conv2.weight False
layer1.1.bn2.weight False
layer1.1.bn2.bias False
layer1.1.conv3.weight False
layer1.1.bn3.weight False
layer1.1.bn3.bias False
layer1.2.conv1.weight False
layer1.2.bn1.weight False
layer1.2.bn1.bias False
layer1.2.conv2.weight False
layer1.2.bn2.weight False
layer1.2.bn2.bias False
layer1.2.conv3.weight False
layer1.2.bn3.weight False
layer1.2.bn3.bias False
layer2.0.conv1.weight False
layer2.0.bn1.weight False
layer2.0.bn1.bias False
layer2.0.conv2.weight False
layer2.0.bn2.weight False
layer2.0.bn2

In [13]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.2, patience=1)

loaders = {'train':train_loader, 'valid':valid_loader, 'test': test_loader}

In [14]:
model = train_model(loaders, model, criterion, optimizer, scheduler, num_epochs=20)

  0%|          | 0/1904 [00:00<?, ?it/s]

Epoch 0/19
----------


 28%|██▊       | 534/1904 [03:20<09:30,  2.40it/s]/home/gp/Documents/made_project/env/lib/python3.8/site-packages/PIL/Image.py:951: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
 44%|████▎     | 831/1904 [05:24<07:00,  2.55it/s]/home/gp/Documents/made_project/env/lib/python3.8/site-packages/PIL/Image.py:951: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
  0%|          | 0/239 [00:00<?, ?it/s]

train Loss: 0.3438 Acc: 0.9096


  0%|          | 0/1904 [00:00<?, ?it/s]

...

train Loss: 0.3073 Acc: 0.9199


  0%|          | 0/1904 [00:00<?, ?it/s]

valid Loss: 0.3080 Acc: 0.9384
Epoch 9/19
----------


  5%|▍         | 90/1904 [00:31<10:31,  2.87it/s]/home/gp/Documents/made_project/env/lib/python3.8/site-packages/PIL/Image.py:951: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
  7%|▋         | 137/1904 [00:48<10:21,  2.84it/s]


KeyboardInterrupt: 

In [15]:
test_acc = []

for data in tqdm(loaders["test"]):
    model.train(False)
    inputs, labels = data
    labels = labels.view(-1)
    
    if use_gpu:
        inputs = inputs.cuda()
        labels = labels.cuda()
        
    inputs = Variable(inputs)
    labels = Variable(labels)

    outputs = model(inputs)
    _, preds = torch.max(outputs.data, 1)
    test_acc.append((labels == preds).cpu().numpy())
    
print("TOP_1 acc: ", np.hstack(test_acc).mean())

100%|██████████| 239/239 [01:14<00:00,  3.22it/s]

TOP_1 acc:  0.9384958363386008


In [16]:
# torch.save(model.state_dict(), "./cache/models/resnet50-new_weights.pth")